In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/invoices/GT Invoices/2084022143.tif
/kaggle/input/invoices/GT Invoices/524483561+-3562_ocr.xml
/kaggle/input/invoices/GT Invoices/11295049_ocr.xml
/kaggle/input/invoices/GT Invoices/2029377724_ocr.xml
/kaggle/input/invoices/GT Invoices/2063216131.tif
/kaggle/input/invoices/GT Invoices/2028729078.tif
/kaggle/input/invoices/GT Invoices/99380808_0809.tif
/kaggle/input/invoices/GT Invoices/0060027778_gt.xml
/kaggle/input/invoices/GT Invoices/ti01410980_gt.xml
/kaggle/input/invoices/GT Invoices/2063207059_7062_gt.xml
/kaggle/input/invoices/GT Invoices/86619053_ocr.xml
/kaggle/input/invoices/GT Invoices/11237705_gt.xml
/kaggle/input/invoices/GT Invoices/2072957946.tif
/kaggle/input/invoices/GT Invoices/2029377724.tif
/kaggle/input/invoices/GT Invoices/2070238500.tif
/kaggle/input/invoices/GT Invoices/0001139626.tif
/kaggle/input/invoices/GT Invoices/0060053761_gt.xml
/kaggle/input/invoices/GT Invoices/2044696237_gt.xml
/kaggle/input/invoices/GT Invoices/91781209.tif
/kaggle/inp

In [3]:

import pandas as pd
import matplotlib.pyplot as plt
import sys
import os

# Image handling libraries
from PIL import Image
import cv2
import pytesseract
from pytesseract import image_to_string



#Text preprocessing libraries
import nltk
import re
from nltk.corpus import stopwords


import logging


In [4]:

def get_image(directory):
    image_path=os.path.join(directory, filename)
    image =cv2.imread(image_path) #read image present in directory
    return image  #calling function
            
def clean_image(img):
    #resizing image
    img = cv2.resize(img, None, fx=1.25, fy=1.25, interpolation=cv2.INTER_CUBIC) #scaling the image
    
    # get grayscale image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #thresholding
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 85, 11)
    
    #call function to extract the text from images
    return img
    
def extract_text(cleaned_image):
    custom_config = r'-l eng --psm 3'
    extracted_Information = pytesseract.image_to_string(cleaned_image,config=custom_config)
    extracted_Information = extracted_Information.replace(",", "")
    extracted_Information = extracted_Information.split("\n")
    return extracted_Information

def clean_text(extracted_text, sentence_id):
    extracted_text =[ text.lower() for text in extracted_text]

    # Remove unicode characters
    extracted_text = [asci.encode('ascii', 'ignore').decode() for asci in extracted_text]
   
    extracted_text = [re.sub("[^-\n$,a-zA-Z.0-9]+", " ", text) for text in extracted_text]
    
    extracted_text = [word for word in extracted_text if word not in stopwords.words('english') ]
    cleaned_text = []
    for sentences in extracted_text:
        sentences = sentences.split(" ")
        for words in sentences:
            if words not in [' ',""]:
                cleaned_text.append(words)
    temp_df =pd.DataFrame()         
    temp_df['Text'] = cleaned_text
    temp_df['SentenceID']=sentence_id
    #print(temp_df)
    return temp_df
    
    


            





In [7]:
directory = r'/kaggle/input/invoices/GT Invoices/'
invoice_data = pd.DataFrame(columns=['Text','SentenceID'])
sentence_id = 0
for filename in os.listdir(directory):
        
        if filename.endswith(".tif") :
            sentence_id +=1
            
            if sentence_id <=8:
                image = get_image(directory)
                cleaned_image = clean_image(image)
                extracted_text = extract_text(cleaned_image)
                invoice_data_df = clean_text(extracted_text,sentence_id)
                invoice_data = invoice_data.append(invoice_data_df,ignore_index=True)




In [ ]:
invoice_data.shape

In [ ]:
invoice_data.to_excel("required2_dataset.xlsx", index=False)